In [14]:
from pydantic import BaseModel, Field
from typing import List, Optional

class Question(BaseModel):
    topic: str = Field(description="Topic of the questions that has been generated")
    question: str = Field(description="Interview question for the candidate")
    answer: str = Field(description="Suggested answer for the question")

In [15]:
"""State management for the index graph."""
from __future__ import annotations
from dataclasses import dataclass, field
from typing import Annotated, Optional

from langchain_core.documents import Document

from shared.state import reduce_docs
from shared.configuration import BaseConfiguration
from dataclasses import dataclass, field
from typing import Optional, TypedDict, List, Dict, Union, Annotated
from langchain.schema import AgentAction, AgentFinish
import operator


class AgentState(TypedDict):

    resume: str = field(
        metadata={
            "description": "The resume of the candidate."
        }
    )
    job_description: str = field(
        metadata={
            "description": "The job description for the role, represented as a string."
        }
    )
    job_title: str = field(
        metadata={
            "description": "The job title for which questions and evaluations are generated."
        }
    )
    company_name: str = field(
        metadata={
            "description": "The name of the company associated with the job description."
        }
    )
    history: Optional[List[Dict[str, str]]] = field(
        default=None,
        metadata={
            "description": "History of previously asked questions and answers as a list of dictionaries."
        }
    )
    agent_out: Union[AgentAction, AgentFinish, None] = field(
        default=None,
        metadata={
            "description": "The output from the agent, which can be an action or a finish signal."
        }
    )
    intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add] = field(
        default_factory=list,
        metadata={
            "description": "A list of intermediate steps taken by the agent, each represented as a tuple of action and description."
        }
    )

In [17]:
"""This "graph" simply exposes an endpoint for a user to upload docs to be indexed."""

import json
import os
from typing import Optional
from langchain_core.runnables import RunnableConfig
from langgraph.graph import END, START, StateGraph
from langchain_core.tools import tool
from langchain_openai.chat_models import AzureChatOpenAI
from langsmith import Client
from dotenv import load_dotenv
from shared.state import reduce_docs
import asyncio
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime
from http.client import HTTPException
import itertools
import random
import uuid
import json

import certifi
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_community.chat_models import ChatAnyscale, BedrockChat
from langchain_openai.chat_models import AzureChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, PromptTemplate, HumanMessagePromptTemplate
from dotenv import load_dotenv
from langchain_core.runnables import RunnableParallel
import os
from urllib3.util import parse_url

load_dotenv()

client = Client()

model = AzureChatOpenAI(temperature=0,
                            seed=42,
                            streaming=True,
                            azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),
                            openai_api_version=os.getenv('AZURE_OPENAI_API_VERSION'),
                            openai_api_key=os.getenv('OPENAI_API_KEY'),
                            deployment_name=os.getenv('AZURE_OPENAI_CHATGPT_DEPLOYMENT'),
                            openai_api_type=os.getenv('OPENAI_API_TYPE')
                            )
def generate_questions(state: list): 
    """Generate interview questions using an LLM.
    Args:
        state (dict): The input state containing required fields.
    Returns:
        dict: A dictionary containing generated questions.
    """
    try:
        print("reachere here")
        # Ensure the state contains all required fields
        required_fields = ["resume", "job_description", "job_title", "company_name", "history"]
        for field in required_fields:
            if field not in state:
                raise ValueError(f"Missing required field: {field}")

        document_prompt = client.pull_prompt("interviewer_prompt")
        chain = (document_prompt | model.with_structured_output(Question))
        response = chain.invoke({
            "resume": state["resume"], 
            "job_description": state["job_description"],
            "job_title": state["job_title"],
            "company_name": state["company_name"],
            "history": state["history"]
        }) 

        print(response)
        
        return {"agent_out": response}
    except Exception as e:
        print("Error:", e)
        return {"agent_out": f"Could not generate questions. Details: {str(e)}"}
@tool
def evaluate_answers(state: AgentState):
    """Generate interview questions using an LLM.
    Args:
        resume (str): The candidate's resume.
        job_description (str): The job description for the role.
        job_title (str): The job title of the role
        company_name (str): The name of the company
        history Optional[str]: History of previously asked questions and answers
    Returns:
        dict: A dictionary containing generated questions categorized into technical, personal, HR, and logical sections.
    """
    try:
        document_prompt = client.pull_prompt("quiz_from_jd")
        chain = (document_prompt | model.with_structured_output(Question))
        response = chain.invoke({"resume_text": state.resume, 
                           "job_description": state.job_description,
                           "job_title": state.job_title,
                           "company_name": state.company_name,
                           "history": state.history})
        return response
    except Exception as e:
        print("Error:", e)
        return {"error": f"Could not generate questions. Details: {str(e)}"}
@tool
   
def evaluate_candidate(state: AgentState):
    """Generate interview questions using an LLM.
    Args:
        resume (str): The candidate's resume.
        job_description (str): The job description for the role.
        job_title (str): The job title of the role
        company_name (str): The name of the company
        history Optional[str]: History of previously asked questions and answers
    Returns:
        dict: A dictionary containing generated questions categorized into technical, personal, HR, and logical sections.
    """
    try:
        document_prompt = client.pull_prompt("quiz_from_jd")
        chain = (document_prompt | model.with_structured_output(Question))
        response = chain.invoke({"resume_text": state.resume, 
                           "job_description": state.job_description,
                           "job_title": state.job_title,
                           "company_name": state.company_name,
                           "history": state.history})
        return response
    except Exception as e:
        print("Error:", e)
        return {"error": f"Could not generate questions. Details: {str(e)}"}
    
def count_questions(state: AgentState):
    """
    Determines whether to finish.

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """

# Define the graph
builder = StateGraph(AgentState)
builder.add_node(generate_questions)
builder.add_node(evaluate_answers)
builder.add_edge(START, "generate_questions")
#builder.add_edge("generate_questions", "evaluate_answers")
#builder.add_edge("evaluate_answers", "generate_questions")
builder.add_edge("generate_questions", END)
# Compile into a graph object that you can invoke and deploy.
graph = builder.compile()
graph.name = "IndexGraph"


agent_state = AgentState(
    resume="John Doe's resume text here...",
    job_description="Job description for a data scientist role...",
    job_title="Data Scientist",
    company_name="Tech Corp",
    history=[{"question": "What are your strengths?", "answer": "Problem-solving and teamwork"}]
)

result = graph.invoke({
    "resume": "John Doe's resume text here...",
    "job_description": "Job description for a data scientist role...",
    "job_title": "Data Scientist",
    "company_name": "Tech Corp",
    "history": [{"question": "What are your strengths?", "answer": "Problem-solving and teamwork"}]
})
agent_state["history"].append({"question": "What is your experience with Python?", "answer": "5 years"})
print(result)


reachere here
Error: Error code: 401 - {'statusCode': 401, 'message': 'Access denied due to invalid subscription key. Make sure to provide a valid key for an active subscription.'}
{'resume': "John Doe's resume text here...", 'job_description': 'Job description for a data scientist role...', 'job_title': 'Data Scientist', 'company_name': 'Tech Corp', 'history': [{'question': 'What are your strengths?', 'answer': 'Problem-solving and teamwork'}], 'agent_out': "Could not generate questions. Details: Error code: 401 - {'statusCode': 401, 'message': 'Access denied due to invalid subscription key. Make sure to provide a valid key for an active subscription.'}", 'intermediate_steps': []}
